In [1]:
import pandas as pd

In [2]:
signals = pd.read_parquet("data/signals.parquet.gzip")
metatags = pd.read_parquet("data/metatags.parquet.gzip")

In [3]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [4]:
articles = intersection(metatags.src.unique(), signals.src.unique() )
signals = signals[signals.src.isin(articles)]
metatags = metatags[metatags.src.isin(articles)]
len(articles)

1111

In [5]:
metatags

,src,keywords,themes,summary,origin,url
1,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...
3,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...
4,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...
5,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...
6,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...
...,...,...,...,...,...,...
1395,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
1396,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
1399,b9d91aca816a4b049d4583a774f886a2,"[Amazonian dark earth, soil, rainforest, ancie...","[Archaeology, Agriculture, Climate Change]",This article explores the importance of Amazon...,20240128,b'https://www.bbc.com/future/article/20240116-...
1400,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

/home/kelu/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
#sentences = list(metatags.summary)
sentences = list(metatags.keywords)
sentences = [", ".join(x) for x in sentences]

In [8]:
from sentence_transformers import util


In [9]:
embeddings = model.encode(sentences, convert_to_tensor=True,show_progress_bar=True)
cosine_scores = util.cos_sim(embeddings, embeddings)
#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

fun, traits, spontaneous, ease, silly, vulnerable, appreciative, joy, attitude, mindset 		 fun, traits, spontaneous, ease, silly, vulnerable, appreciative, joy, attitude, mindset 		 Score: 1.0000
trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 trial, digital access, FT.com, Standard Digital, Premium Digital, global news, analysis, expert opinion, Lex, business themes 		 Score: 1.0000
keyword0 for keyword in top_keywords 		 keyword0 for keyword in top_keywords 		 Score: 1.0000
déploiement, intelligence artificielle, modèles de langage, chaîne de traitement numérique, compétences humaines spécialisées, connectivité de haute qualité, expérience utilisateur, stockage performant, capacités de calcul haute performance, pile technologique applicative 		 déploiement, intelligence artificielle, modèles de langage, chaîne de traitement numérique, compétences humaines spécialisées, connectivité de haute qualité, expé

It takes 4 mins for 1000 articles. Approx.

In [10]:
dfSent = pd.DataFrame([sentences, list(metatags.src)]).T
dfSent.columns = ["txt","src"]
dfSent.to_parquet("pages/dfSentences.parquet.gzip",compression="gzip")
dfSent

,txt,src
0,"AI panic, classrooms, education system, adapt,...",ea74b217d36ec570bb432fb5aa679090
1,"backdoor, cybersecurity researchers, encrypted...",841546c0efc4c82f0aabc545a47a09e1
2,"Cybersecurity Researchers, Sandbox Satellite, ...",f6637d1fa0e3fe0e1e94bea8b3338ef9
3,"regulating artificial intelligence, safeguards...",58612e45effab1e47df9a86b14dfae85
4,"Chat-GPT, Python, Knowledge Graph, Neo4j, arti...",7ff6239e13a43e856fa36887902c5ede
...,...,...
1106,"hanging out, book, Sheila Liming, crisis, isol...",edbb7336dfb26098bcc966ca17c074b9
1107,"insomniacs, sleeping assistants, Taobao, Chine...",3fbb11d6e949d1e662aa6a146bb6cda0
1108,"Amazonian dark earth, soil, rainforest, ancien...",b9d91aca816a4b049d4583a774f886a2
1109,"U.S. government, Nvidia, high-performance comp...",b79a2baa87b68283198416791b93bce4


In [11]:
dfPairs = pd.DataFrame(pairs)
dfPairs["score"] = dfPairs["score"].astype(float)
dfPairs["i"] = dfPairs["index"].apply(lambda x: x[0])
dfPairs["j"] = dfPairs["index"].apply(lambda x: x[1])
dfPairs.to_parquet("pages/dfPairs.parquet.gzip",compression="gzip")
dfPairs

,index,score,i,j
0,"[547, 713]",1.000001,547,713
1,"[36, 1101]",1.000000,36,1101
2,"[405, 1001]",1.000000,405,1001
3,"[766, 1087]",1.000000,766,1087
4,"[788, 866]",1.000000,788,866
...,...,...,...,...
616600,"[299, 303]",-0.040947,299,303
616601,"[377, 973]",-0.050092,377,973
616602,"[488, 973]",-0.056254,488,973
616603,"[299, 960]",-0.057838,299,960


In [12]:
def getClosest(i):
    RES = dfPairs[ ((dfPairs["i"] == i) & (dfPairs["j"] != i)) \
      + ((dfPairs["j"] == i) & (dfPairs["i"] != i))]

    closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])
    return [metatags.iloc[x].src for x in closest]

In [13]:
i = 3
RES = dfPairs[ ((dfPairs["i"] == i) & (dfPairs["j"] != i)) ]

closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])
closest

/tmp/ipykernel_1592284/1472497355.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])


[905, 1051, 306, 152, 575, 902, 131, 276, 49, 394]

In [14]:
metatags = metatags.reset_index()
metatags = metatags.drop("index",axis=1)
metatags

,src,keywords,themes,summary,origin,url
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...
...,...,...,...,...,...,...
1106,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
1107,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
1108,b9d91aca816a4b049d4583a774f886a2,"[Amazonian dark earth, soil, rainforest, ancie...","[Archaeology, Agriculture, Climate Change]",This article explores the importance of Amazon...,20240128,b'https://www.bbc.com/future/article/20240116-...
1109,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [15]:
metatags["closest"] = ""
for ix, row in metatags.iterrows():
    row["closest"] = getClosest(ix)
metatags

/tmp/ipykernel_1592284/2972742187.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  closest = list(RES.sort_values(by="score", ascending=False )["j"][:10])


,src,keywords,themes,summary,origin,url,closest
0,ea74b217d36ec570bb432fb5aa679090,"[AI panic, classrooms, education system, adapt...","[Impact of AI in Education, Adapting to AI in ...",The introduction of AI in classrooms has spark...,20230416,b'https://www.oneusefulthing.org/p/the-future-...,"[b30a4282af9e53ca673438a8223d9525, 3aaa975f44b..."
1,841546c0efc4c82f0aabc545a47a09e1,"[backdoor, cybersecurity researchers, encrypte...","[cybersecurity, encryption, vulnerabilities]",A group of cybersecurity researchers has disco...,20230810,b'https://www.vice.com/en/article/4a3n3j/backd...,"[dabd88039518d1eb97139f16625eeec8, f6637d1fa0e..."
2,f6637d1fa0e3fe0e1e94bea8b3338ef9,"[Cybersecurity Researchers, Sandbox Satellite,...","[Satellite security, Hack-a-Sat competition, C...",Researchers have launched a satellite called M...,20230708,b'https://spaceref.com/newspace-and-tech/cyber...,"[ae919b08f9b58de655abd2fea03c1169, 0b6d917158c..."
3,58612e45effab1e47df9a86b14dfae85,"[regulating artificial intelligence, safeguard...","[Regulating AI, Ethical concerns, Technologica...",The text discusses the challenges of regulatin...,20231209,b'https://thegeneralist.substack.com/p/life-in...,"[2cfdc6fba6f11e089c2df3cda1604174, df59e2cf338..."
4,7ff6239e13a43e856fa36887902c5ede,"[Chat-GPT, Python, Knowledge Graph, Neo4j, art...","[Artificial Intelligence, Natural Language Pro...",This article discusses how to use Chat-GPT and...,20290911,b'https://towardsdatascience.com/how-to-use-ch...,"[e0f6ae75e034df0a32044fe8a9075673, 70645a8fbb8..."
...,...,...,...,...,...,...,...
1106,edbb7336dfb26098bcc966ca17c074b9,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...,"[edbb7336dfb26098bcc966ca17c074b9, edbb7336dfb..."
1107,3fbb11d6e949d1e662aa6a146bb6cda0,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...,"[3fbb11d6e949d1e662aa6a146bb6cda0, 3fbb11d6e94..."
1108,b9d91aca816a4b049d4583a774f886a2,"[Amazonian dark earth, soil, rainforest, ancie...","[Archaeology, Agriculture, Climate Change]",This article explores the importance of Amazon...,20240128,b'https://www.bbc.com/future/article/20240116-...,"[b9d91aca816a4b049d4583a774f886a2, b9d91aca816..."
1109,b79a2baa87b68283198416791b93bce4,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...,"[b79a2baa87b68283198416791b93bce4, b79a2baa87b..."


In [16]:
metatags.to_parquet("pages/metatags_with_closest.parquet.gzip", compression="gzip")

In [17]:
metatags = pd.read_parquet("pages/metatags_with_closest.parquet.gzip")

In [18]:
signals

,Signal,Change,10y horizon,Driving force,src
0,AI's impact on education ...,Adaptation and improvement,"Integration of AI in classrooms, AI tutoring,...",Potential for improved learning and teaching ...,ea74b217d36ec570bb432fb5aa679090
1,AI cheating in classrooms ...,Widespread and undetectable,"Continued AI cheating, advancements in AI tec...",Easier access to AI technology and the desire...,ea74b217d36ec570bb432fb5aa679090
2,AI transforming classrooms ...,Fundamental transformation,"Flipped classrooms, personalized learning, en...",Potential to enhance learning experience and ...,ea74b217d36ec570bb432fb5aa679090
3,Concerns about the impact of calculators on e...,Transition and adaptation,"Calculators integrated into curriculum, chang...",Desire for increased student engagement and p...,ea74b217d36ec570bb432fb5aa679090
4,Potential impact of AI on literacy skills ...,Uncertain,"Potential decline in literacy skills, relianc...",Potential concern for declining writing skill...,ea74b217d36ec570bb432fb5aa679090
...,...,...,...,...,...
6863,Middle East countries impacted by restrictions,Limitation on access to high-performance GPUs,Shift in AI technology supply chains ...,Concerns over technology diversion to China ...,b79a2baa87b68283198416791b93bce4
6864,Saudi Arabia and UAE strengthening AI prowess,Increased focus on AI development,Greater investment and collaboration in AI ...,Desire to advance AI capabilities ...,b79a2baa87b68283198416791b93bce4
6865,U.S. concern over technology diversion to China,Efforts to prevent Chinese AI development,Stricter regulations and controls on technology,Protecting domestic AI industry and capabilit...,b79a2baa87b68283198416791b93bce4
6866,Nvidia working with U.S. government,Collaboration to address licensing issue,Resolution of licensing requirements ...,Maintaining business operations and revenue s...,b79a2baa87b68283198416791b93bce4


In [19]:
titles = pd.read_parquet("titles.parquet.gzip")
titles
TITLES = {}
for ix, row in titles.iterrows():
    TITLES[row["src"]] =  row["title"] 

In [20]:
titles

,src,title
0,cc291c4e68ac16cdef2b1a424f29ef20,Co-Intelligence: Navigating the AI Revolution
1,ea74b217d36ec570bb432fb5aa679090,The Impact of AI on Education
2,d07a9512bded6c5e9871608ec5edefec,Crafting Better Prompts for Improved Results
3,841546c0efc4c82f0aabc545a47a09e1,Researchers Uncover Backdoor in Encrypted Radi...
4,f6637d1fa0e3fe0e1e94bea8b3338ef9,Hack-a-Sat: Researchers Launch Sandbox Satelli...
...,...,...
1405,e8b7faf708568f9f39abe04b778c4631,The Impact of ChatGPT on Efficiency and Inform...
1406,f1011f7f464bfcb22dcb75731bd0090f,Advancing GPT Models through Prompting Strategies
1407,b9d91aca816a4b049d4583a774f886a2,The Ancient Agricultural Secret of Amazonian D...
1408,b79a2baa87b68283198416791b93bce4,US Restricts Nvidia GPU Sales to Thwart China'...


In [21]:
metatags["kwsl"] = metatags["keywords"].apply(lambda lst: [x.lower().replace('"','') for x in lst])


In [22]:
metatags[metatags.src == "1ef0f968ac542dcb76eef026bdb6791e"]

,src,keywords,themes,summary,origin,url,closest,kwsl
675,1ef0f968ac542dcb76eef026bdb6791e,"[Generative AI, AI detectors, AI writing, chea...","[Generative AI, AI detection and limitations, ...","In this article, the author discusses several ...",20231203,b'https://www.oneusefulthing.org/p/what-people...,"[e1fbb09ec5e66a8a6d4eff2126eefb40, 1ef0f968ac5...","[generative ai, ai detectors, ai writing, chea..."


In [23]:
for ix, row in metatags.iterrows():
    try:
        ID = row["src"]
        TXT = "# __"+TITLES[ID] +"__, from (["+row.origin+"](https://kghosh.substack.com/p/"+row.origin+").)\n\n"
        TXT += "__[External link]("+str(row.url)[2:-1]+")__\n\n"
        TXT += "\n\n## Summary\n\n"
        TXT += row["summary"] + "\n\n## Keywords\n\n"
        TXT += "* "+"\n* ".join(row["keywords"])
        TXT += "\n\n## Themes\n\n"
        TXT += "* "+"\n* ".join(row["themes"])
        TXT += "\n\n## Signals\n\n"    
        SIG = signals[signals.src == ID]
        TXT += SIG[list(SIG.columns)[:-1]].to_markdown(index=False)
        if 1:
            TXT += "\n\n## Closest\n\n"
            TXT += "* "+"\n* ".join([ "["+TITLES[x]+"]("+x+")" for x in row["closest"]])
        with open("docs/"+ID+".md", "w") as f:
            f.write(TXT) 
    except:
        print("Error with",row["src"])


In [24]:
ID

'69637dcd83c48ebde0610a61a27b1989'

In [25]:
allkw = []
for kw in metatags.themes:# keywords: #as an alternative
    allkw = allkw+list(kw)
KW = list(set(allkw))
KW = list(set([x.capitalize().replace('"','') for x in KW]))
KW = [x for x in KW if not x.isnumeric()]
KW.sort()
print(len(KW))
KW

2752


['2024 problem in logistics',
 '3d imaging',
 '3d printing technology in construction',
 'Academic vs. intellectual',
 'Access plans',
 'Access to culture',
 'Access to fresh groceries',
 'Access to tools and resources',
 'Accessibility and applications of ai to everyday life',
 'Accountability',
 'Accounting',
 'Activation',
 'Activism',
 'Activisme écologique',
 'Activity partners',
 'Adaptation of education system',
 'Adaptation to changing needs',
 'Adapting to ai in the classroom',
 'Adaptive reuse apartments',
 'Adding',
 'Additional revenue',
 'Adhd in the workplace',
 'Adoption of generative ai',
 'Adoption of generative ai in business',
 'Advanced bionic limb',
 'Advanced packaging',
 'Advanced technology',
 'Advancements in ai',
 'Advancements in civil engineering',
 'Advancements in generative ai',
 'Advancements in watch technology',
 'Advances in artificial intelligence',
 'Advances in language models',
 'Advances in medical imaging',
 'Advantages of ai',
 'Advantages of a

In [26]:
import string
LETTERS = string.ascii_uppercase
LETTERS

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [27]:
INDEX = "Index \n\n"

for letter in LETTERS:
    kws = [x for x in KW if x[0] == letter]

    if len(kws):
        INDEX += "\n\n## Letter '"+letter.upper()+"' \n\n"
        for kw in kws:
            INDEX += "\n* __"+kw+"__: "
            mask = metatags.themes.apply(lambda x: any(item for item in [kw] if item in [X.capitalize() for X in x]))
            lst = metatags[mask].src.tolist()
            INDEX += ", ".join([ "["+TITLES[x]+"]("+x+")" for x in lst ])
INDEX += "\n\n## Others\n\n"
kws = [x for x in KW if x[0].isnumeric()]
for kw in kws:
    INDEX += "\n* __"+kw+"__: "
    mask = metatags.kwsl.apply(lambda x: any(item for item in [kw] if item in [X.capitalize() for X in x]))
    lst = metatags[mask].src.tolist()
    INDEX += ", ".join([ "["+TITLES[x]+"]("+x+")" for x in lst ])

In [28]:
with open("docs/index.md", "w") as f:
    f.write(INDEX) 

In [29]:
INDEX

'Index \n\n\n\n## Letter \'A\' \n\n\n* __Academic vs. intellectual__: [The Difference Between Academics and Intellectuals](f07df9d85e95d88e18345a39166c0a53)\n* __Access plans__: [FT.com Trial and Subscription Information](34e1369572f1241d4a54e63cee2a4565)\n* __Access to culture__: [Cultural Therapy: Museum Prescriptions for Mental Health](3c807d80ce080c015f1fc14086b7a09e)\n* __Access to fresh groceries__: [Uber Eats to Accept Food Stamps and Healthcare Benefits for Grocery Delivery](01100fc1dbc0489489a31e12750bca78)\n* __Access to tools and resources__: [American Farm Bureau and John Deere Sign Agreement on Equipment Repair](26ae831dfd3721235ac0eacb85e4a0e5)\n* __Accessibility and applications of ai to everyday life__: [AI Superpowers: Unleashing Entrepreneurship in 2023](a40580730388900810b4496ff9891dc9)\n* __Accountability__: [Tracking Openness in ChatGPT Generators](dad9dbd97cc50604963911f16fbb27aa)\n* __Accounting__: [The Triumph of the Bean Counters](74640ec9669a4ca9964b6a3aca93f3